In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle

In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Se carga el set de entrenamiento

In [4]:
prop = pickle.load(open("../../datos/Data.p","rb"))

In [5]:
proper = pickle.load(open("../../datos/propiedadesPruebaConColTransGas.p", "rb"))

In [6]:
propTOT = pd.merge(prop, proper, how="outer")

In [7]:
#cortamos por precios
propTOT = propTOT[(propTOT['price_usd_per_m2']>150) & (propTOT['price_usd_per_m2']<8000)]

In [ ]:
#completo las superficies totales con las cubiertas
propTOT['surface_total_in_m2'] = propTOT.apply(lambda row: row[17] if(np.all(pd.isnull(row[16]))) else row[16], axis=1)

In [ ]:
#se formatea lat y lon
p = propTOT[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]
p.dropna(inplace = True)

p['lat'] = p.apply(lambda row: row[4][0], axis=1)

p['lon'] = p.apply(lambda row: row[4][1], axis=1)

p = p[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

### Se le asigna un valor numerico al tipo de propiedad

In [13]:
def asignarPT( p ):
    if p == 'apartment':
        return 0
    if p == 'house':
        return 1
    if p == 'store':
        return 2
    return 3

In [14]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

### Se le asigna un valor a los barrios segun el analisis de grupos hecho en el tp1

In [15]:
def asignarPV(name, dic):
    return dic[name]   

In [16]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [17]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)*10
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [18]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [19]:
pEnt = p[['property_type','surface_total_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value']]

In [20]:
len(pEnt)

77492

# Se carga el set de Prueba

In [21]:
proper = pickle.load(open("porpiedadesPruebaConColTransGas.p", "rb"))

In [22]:
len(proper)

14166

In [23]:
#se formatea lat y lon
p = proper[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

p['lat'] = p.apply(lambda row: row[4][0] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p['lon'] = p.apply(lambda row: row[4][1] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [24]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

In [25]:
HGdics

{' Country Maschwitz Club': 2686.926583591594,
 ' los alamos': 1223.155562884655,
 'Abasto': 691.0196765958824,
 'Abril Club de Campo': 4093.402187612541,
 'Acacias Blancas': 3318.9518010064007,
 'Acassuso': 751.333799018616,
 'Adrogu\xc3\xa9': 1614.3465205401174,
 'Aeropuerto Internacional Ezeiza': 2396.0349238804647,
 'Agronom\xc3\xada': 590.3756834347769,
 'Albanueva Barrio Cerrado': 228.5512379584443,
 'Aldo Bonzi': 3580.117344108276,
 'Alejandro Korn': 3429.364364962655,
 'Alejandro Peti\xc3\xb3n': 3761.3212422003116,
 'Almagro': 469.9637057632352,
 'Almirante Brown': 1945.1723673074127,
 'Altamira': 821.5656244679294,
 'Altos de Hudson II': 721.3018504717331,
 'Altos de Manzanares 1 y 2': 1834.983296938517,
 'Altos de Matheu': 3288.815428986968,
 'Altos del Golf': 3409.2692309743475,
 'Altos del Pilar': 459.8924596950678,
 'Armenia Country Club': 2195.565466521068,
 'Avellaneda': 1423.885780458288,
 'Bah\xc3\xada del Sol': 309.0817065650197,
 'Balvanera': 811.4678581170713,
 'Ban

In [26]:
def asignarPVsegunEnt(name):
    if(HGdics.has_key(name)):
        return HGdics[name]
    return np.nan

In [27]:
#hay que armar los hoodGropus segun los resultados del set de Entrenamiento
p["place_value"] = p.apply(lambda x: asignarPVsegunEnt(x[1]), axis=1)

In [28]:
#completo las superficies totales con las cubiertas
p['surface_total_in_m2'] = p.apply(lambda row: row[3] if(np.all(pd.isnull(row[2]))) else row[2], axis=1)

In [29]:
pPru = p[['property_type','surface_total_in_m2',\
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value']]

# PRUEBA Random Forest Regresivo

In [30]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(pEnt[['property_type','surface_total_in_m2',
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos','Cant_ParadasTransporte',
        'place_value']],pEnt[['price_usd_per_m2']],test_size=0.0)

In [31]:
#definir el modelo a probar
model = RandomForestRegressor(n_estimators=10, max_features='auto', min_samples_leaf=1, min_samples_split=4, bootstrap=True)
n = 10

#fit model on training dataset
model.fit(Xtrn, Ytrn['price_usd_per_m2'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [31]:
pPru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 8 columns):
property_type                14166 non-null int64
surface_total_in_m2          13392 non-null float64
lat                          12586 non-null float64
lon                          12586 non-null float64
Cant_ColeYUniv               12586 non-null float64
Cant_LocalesGastronomicos    12586 non-null float64
Cant_ParadasTransporte       12586 non-null float64
place_value                  14157 non-null float64
dtypes: float64(7), int64(1)
memory usage: 885.4 KB


In [33]:
from sklearn.preprocessing import Imputer
#hay que llenar el resto de los valores nan

# Create our imputer to replace missing values with the mean e.g.
imp = Imputer(missing_values=np.nan, strategy='mean', axis=0)
imp = imp.fit(Xtrn)

pPru_imp = imp.transform(pPru)

In [34]:
pPru['price_usd_per_m2'] = model.predict(pPru_imp)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
pPru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 9 columns):
property_type                14166 non-null int64
surface_total_in_m2          13392 non-null float64
lat                          12586 non-null float64
lon                          12586 non-null float64
Cant_ColeYUniv               12586 non-null float64
Cant_LocalesGastronomicos    12586 non-null float64
Cant_ParadasTransporte       12586 non-null float64
place_value                  14156 non-null float64
price_usd_per_m2             14166 non-null float64
dtypes: float64(8), int64(1)
memory usage: 996.1 KB


In [36]:
pPru['price_usd'] = pPru.apply(lambda row: row[8]*206.87862809 if(np.all(pd.isnull(row[1]))) else row[1]*row[8], axis=1)

In [37]:
result = pd.DataFrame()

In [38]:
result["id"] = proper["id"]

In [39]:
result["price_usd"] = pPru['price_usd']

In [40]:
result.head()

id      price_usd
0  2316505  205542.281082
1  2320880  213346.722214
2  2321742  221229.082068
3  2276796   94887.418245
4  2330364   65717.389440

In [41]:
result.to_csv(index=False, path_or_buf="results2PhiAlpha.csv")

In [42]:
result["real_price_usd"] = proper['price_aprox_usd']